In [68]:
import requests
import pandas as pd
def get_fred_series_observations(series_id, api_key):
    # Endpoint for series observations
    base_url = "https://api.stlouisfed.org/fred/series/observations"
    params = {
        "series_id": series_id,
        "api_key": api_key,
        "file_type": "json"
    }
    response = requests.get(base_url, params=params)
    return response.json()

api_key = 'hyef8c3af7f7bebd62ffff5b460d66375axs'
series_id = 'CPIAUCSL'

# Fetch the data points for the series
data = get_fred_series_observations(series_id, api_key)

# Check if observations are in the response and create a DataFrame
if 'observations' in data:
    df = pd.DataFrame(data['observations'])
    df = df[['date', 'value']]  # Select only the 'date' and 'value' columns
# Convert 'value' column to float64
df['value'] = df['value'].astype('float64')
df['value'] = (df['value'].pct_change(periods=12) * 100).round(2)  # Calculate percent change
df = df.dropna(subset=['value'])

In [ ]:
# Set the 'year' column as the index
df.set_index('date', inplace=True)
df


In [70]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from keras.models import Sequential
from keras.layers import LSTM, Dense
# Normalize the data
scaler = MinMaxScaler()
data_scaled = scaler.fit_transform(df)

In [71]:
import numpy as np
# Split data into train and test sets
train_size = int(len(data_scaled) * 0.8)
train_data, test_data = data_scaled[:train_size], data_scaled[train_size:]

# Create sequences for LSTM training
def create_sequences(df, seq_length):
    X, y = [], []
    for i in range(len(df) - seq_length):
        X.append(df[i:i+seq_length])
        y.append(df[i+seq_length])
    return np.array(X), np.array(y)

seq_length = 1  # Length of sequences for LSTM
X_train, y_train = create_sequences(train_data, seq_length)
X_test, y_test = create_sequences(test_data, seq_length)

In [ ]:
from sklearn.model_selection import ParameterGrid
from keras.callbacks import EarlyStopping
from keras.models import Sequential
from keras.layers import LSTM, Dense

# Define a parameter grid
param_grid = {
    'lstm_units': [20, 50, 100],
    'epochs': [50, 100, 150],
    'batch_size': [16, 32, 64]
}

# Initialize early stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=10)

best_mse = float('inf')
best_params = {}

# Iterate over all combinations of parameters
for params in ParameterGrid(param_grid):
    # Build the LSTM model
    model = Sequential()
    model.add(LSTM(units=params['lstm_units'], input_shape=(X_train.shape[1], X_train.shape[2])))
    model.add(Dense(units=1))
    model.compile(optimizer='adam', loss='mse')

    # Train the model with early stopping and validation split
    model.fit(X_train, y_train, epochs=params['epochs'], batch_size=params['batch_size'], callbacks=[early_stopping], validation_split=0.2)

    # Evaluate the model on the test set
    mse = model.evaluate(X_test, y_test)
    if mse < best_mse:
        best_mse = mse
        best_params = params

# Print the best Mean Squared Error and corresponding parameters
print(f"Best MSE: {best_mse}")
print(f"Best Params: {best_params}")


In [ ]:
# Build and train the LSTM model
# Best params from batch : 16 epochs :150 and units:100
model = Sequential()
model.add(LSTM(units=100, input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(Dense(units=1))
model.compile(optimizer='adam', loss='mse')
model.fit(X_train, y_train, epochs=150, batch_size=16)

# Predictions
train_predict = model.predict(X_train)
test_predict = model.predict(X_test)

# Inverse transform predictions
train_predict = scaler.inverse_transform(train_predict)
y_train = scaler.inverse_transform(y_train)
test_predict = scaler.inverse_transform(test_predict)
y_test = scaler.inverse_transform(y_test)

In [73]:
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import plotly.express as px
# Create a DataFrame for plotting
plot_data = pd.DataFrame({
    'Date': np.concatenate((df.index[seq_length:seq_length+len(train_predict)],
                            df.index[seq_length+len(train_predict):seq_length+len(train_predict)+len(test_predict)])),
    'Predicted CPI': np.concatenate((train_predict.flatten(), test_predict.flatten())),
    'Actual CPI': np.concatenate((y_train.flatten(), y_test.flatten()))
})

# Create an interactive line plot using Plotly Express
fig = px.line(plot_data, x='Date', y=['Predicted CPI', 'Actual CPI'], title='CPI Forecasting with LSTM')
fig.update_layout(xaxis_title='Date', yaxis_title='CPI', legend_title='Data')
fig.show()

In [ ]:
# For the training data
train_years = df.index[seq_length:len(y_train) + seq_length]
# For the testing data
test_years = df.index[len(y_train) + seq_length:len(y_train) + seq_length + len(y_test)]

# Combine years
combined_years = np.concatenate((train_years, test_years))

# Creating a DataFrame for the actual and predicted values
results = pd.DataFrame({
    'Year': combined_years,
    'Actual': np.concatenate((y_train.flatten(), y_test.flatten())),
    'Predicted': np.concatenate((train_predict.flatten(), test_predict.flatten()))
})

# Display the DataFrame
print(results.tail(13))  # Shows the first few rows

In [ ]:
# Assuming you have already trained your model and have the scaler
# Forecasted value for the new month

# Prepare new data (here, we are using the last available data point)
new_data = df.iloc[-seq_length:]['value'].values.reshape(-1, 1)

# Scale the new data
new_data_scaled = scaler.transform(new_data)

# Reshape for LSTM input
new_data_scaled = new_data_scaled.reshape(1, seq_length, 1)

# Forecast
new_month_prediction_scaled = model.predict(new_data_scaled)

# Inverse transform the prediction
new_month_prediction = scaler.inverse_transform(new_month_prediction_scaled)

print(f"Forecasted value for the new month: {new_month_prediction[0][0]}")


In [ ]:
# Assuming your model and scaler are already defined and trained
# Forecasted value for the new 3 months:

# Number of months to forecast
months_to_forecast = 3

# Start with the last available sequence
new_data = df.iloc[-seq_length:]['value'].values.reshape(-1, 1)

# Scale the initial sequence
new_data_scaled = scaler.transform(new_data)

# Iteratively forecast the next months
for _ in range(months_to_forecast):
    # Reshape for LSTM input
    lstm_input = new_data_scaled.reshape(1, seq_length, 1)

    # Forecast the next month
    next_month_prediction_scaled = model.predict(lstm_input)

    # Append the prediction for the next iteration
    new_data_scaled = np.append(new_data_scaled, next_month_prediction_scaled)[-seq_length:]

    # Inverse transform the prediction
    next_month_prediction = scaler.inverse_transform(next_month_prediction_scaled.reshape(-1, 1))

    print(f"Forecasted value for the next month: {next_month_prediction[0][0]}")


In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import plotly.express as px

# Assuming 'model' and 'scaler' are your trained LSTM model and MinMaxScaler, respectively

# Function to forecast the next month
def forecast_next_month(model, scaler, last_data_scaled, seq_length):
    lstm_input = last_data_scaled.reshape(1, seq_length, 1)
    next_month_prediction_scaled = model.predict(lstm_input)
    next_month_prediction = scaler.inverse_transform(next_month_prediction_scaled.reshape(-1, 1))
    return next_month_prediction[0][0]

# Start with the last available sequence for forecasting
last_data = df.iloc[-seq_length:]['value'].values.reshape(-1, 1)
last_data_scaled = scaler.transform(last_data)

# Forecast the next three months
forecasted_values = []
for _ in range(3):
    next_month_prediction = forecast_next_month(model, scaler, last_data_scaled, seq_length)
    forecasted_values.append(next_month_prediction)
    # Update last_data_scaled for the next iteration
    next_month_scaled = scaler.transform(np.array([[next_month_prediction]]))
    last_data_scaled = np.append(last_data_scaled, next_month_scaled)[-seq_length:]

# Get the last date from your DataFrame and generate new dates for the forecasted months
last_date = pd.to_datetime(df.index[-1])
new_dates = [last_date + pd.DateOffset(months=i) for i in range(1, 4)]

# Append the forecasted values and new dates to the plot_data DataFrame
for date, value in zip(new_dates, forecasted_values):
    plot_data = plot_data.append({'Date': date, 'Predicted CPI': value, 'Actual CPI': np.nan}, ignore_index=True)

# Plot the extended data
fig = px.line(plot_data, x='Date', y=['Predicted CPI', 'Actual CPI'], title='Extended CPI Forecast with LSTM')
fig.update_layout(xaxis_title='Date', yaxis_title='CPI', legend_title='Data')
fig.show()


In [79]:
# Plot the extended data
fig = px.line(plot_data, x='Date', y=['Predicted CPI', 'Actual CPI'], title='Extended CPI Forecast with LSTM')
fig.update_layout(xaxis_title='Date', yaxis_title='CPI', legend_title='Data')
fig.show()